In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print(tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
from tensorflow.python.client import device_lib

local_devices = device_lib.list_local_devices()
gpus = [device.name for device in local_devices if device.device_type == 'GPU']
print("Available GPUs:", gpus)

Available GPUs: ['/device:GPU:0']


In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
from pyswarm import pso

In [3]:
data = pd.read_csv('training.csv', header=None)
data.head()

,0,1,2,3,4
0,-0.145,-0.145,-0.145,-0.145,-0.145
1,-0.145,-0.145,-0.145,-0.145,-0.145
2,-0.145,-0.145,-0.135,-0.145,-0.120
3,-0.135,-0.145,-0.160,-0.155,-0.150
4,-0.160,-0.155,-0.175,-0.180,-0.160


In [4]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(53.315155, shape=(), dtype=float32)


In [7]:
from tensorflow.python.client import device_lib

local_devices = device_lib.list_local_devices()
gpus = [device.name for device in local_devices if device.device_type == 'GPU']
if gpus:
    print("Keras menggunakan GPU:", gpus)
else:
    print("Keras menggunakan CPU")

Keras menggunakan GPU: ['/device:GPU:0']


In [14]:
tf.debugging.set_log_device_placement(True)
index = 0

# Membuat dataset TensorFlow dari data Anda
# Menggunakan data yang telah Anda baca dari berkas CSV
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

# Menambahkan metode prefetch() ke dataset
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Fungsi objektif untuk PSO (menggunakan MSE sebagai objektif)
def objective_function(params):
    global index
    print(f'Proses ke-{index}')
    index += 1

    # Membangun model jaringan saraf tiruan
    model = keras.Sequential([
        keras.layers.Dense(10, input_dim=data.shape[1] - 1, activation='relu'),
        keras.layers.Dense(1, activation='linear')
    ])
    
    # Set parameter berdasarkan hasil optimasi PSO
    num_input_units = data.shape[1] - 1
    num_hidden_units = 10
    num_output_units = 1
    
    num_weights_layer1 = num_input_units * num_hidden_units
    num_bias_layer1 = num_hidden_units
    num_weights_layer2 = num_hidden_units * num_output_units
    num_bias_layer2 = num_output_units
    
    weights_layer1 = params[:num_weights_layer1].reshape((num_input_units, num_hidden_units))
    bias_layer1 = params[num_weights_layer1:num_weights_layer1 + num_bias_layer1]
    weights_layer2 = params[-num_weights_layer2 - num_bias_layer2:-num_bias_layer2].reshape((num_hidden_units, num_output_units))
    bias_layer2 = params[-num_bias_layer2:]
    
    model.layers[0].set_weights([weights_layer1, bias_layer1])
    model.layers[1].set_weights([weights_layer2, bias_layer2])
    
    # Kompilasi model
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # Latih model
    print('Fit')
    model.fit(dataset, epochs=10, verbose=1)
    print('Fit End')
    
    # Evaluasi model dan mengembalikan nilai yang ingin dioptimasi
    loss = model.evaluate(X, y, verbose=0)
    return loss

# Batasan parameter untuk PSO
num_input_units = data.shape[1] - 1
num_hidden_units = 10
num_output_units = 1

num_weights_layer1 = num_input_units * num_hidden_units
num_bias_layer1 = num_hidden_units
num_weights_layer2 = num_hidden_units * num_output_units
num_bias_layer2 = num_output_units

num_parameters = num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2
lb = [-1.0] * num_parameters  # Batas bawah parameter
ub = [1.0] * num_parameters   # Batas atas parameter

# Jalankan PSO untuk mengoptimasi parameter jaringan
best_params, _ = pso(objective_function, lb, ub, swarmsize=10, maxiter=25)

# Bangun model jaringan saraf tiruan dengan parameter terbaik
best_model = keras.Sequential([
    keras.layers.Dense(10, input_dim=num_input_units, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])
weights_layer1 = best_params[:num_weights_layer1].reshape((num_input_units, num_hidden_units))
bias_layer1 = best_params[num_weights_layer1:num_weights_layer1 + num_bias_layer1]
weights_layer2 = best_params[-num_weights_layer2 - num_bias_layer2:-num_bias_layer2].reshape((num_hidden_units, num_output_units))
bias_layer2 = best_params[-num_bias_layer2:]
best_model.layers[0].set_weights([weights_layer1, bias_layer1])
best_model.layers[1].set_weights([weights_layer2, bias_layer2])

Proses ke-0
Fit
Epoch 1/10
862/862 [==============================] - 6s 7ms/step - loss: 0.0181
Epoch 2/10
862/862 [==============================] - 7s 8ms/step - loss: 0.0011
Epoch 3/10
862/862 [==============================] - 5s 6ms/step - loss: 5.9905e-04
Epoch 4/10
862/862 [==============================] - 5s 5ms/step - loss: 3.7790e-04
Epoch 5/10
862/862 [==============================] - 5s 5ms/step - loss: 2.6085e-04
Epoch 6/10
862/862 [==============================] - 5s 5ms/step - loss: 2.0023e-04
Epoch 7/10
862/862 [==============================] - 5s 5ms/step - loss: 1.7363e-04
Epoch 8/10
862/862 [==============================] - 5s 5ms/step - loss: 1.6075e-04
Epoch 9/10
862/862 [==============================] - 6s 7ms/step - loss: 1.5316e-04
Epoch 10/10
862/862 [==============================] - 6s 7ms/step - loss: 1.4783e-04
Fit End
Proses ke-1
Fit
Epoch 1/10
862/862 [==============================] - 7s 8ms/step - loss: 0.2783
Epoch 2/10
862/862 [================

In [15]:
# Simpan model terbaik ke dalam berkas H5
best_model.save("ann_pso.h5")

In [18]:
model = tf.keras.models.load_model('ann_pso.h5')
model.predict([[0.145,-0.145,-0.145,-0.145]])

1/1 [==============================] - 0s 85ms/step


array([[-0.6241075]], dtype=float32)